## Reference

https://medium.com/@akriti.upadhyay/implementing-rag-with-langchain-and-hugging-face-28e3ea66c5f7#:%7E:text=Introduction.%20Retrieval%20Augmented%20Generation%20(RAG)%20is%20a%20pattern%20that%20works



https://medium.com/data-science-at-microsoft/evaluating-llm-systems-metrics-challenges-and-best-practices-664ac25be7e5


https://www.datacamp.com/blog/llm-evaluation

https://www.astera.com/type/blog/nlp-information-extraction/

https://outsystems-muc-prod.infineon.com/GPT4IFX/Share?GUID=ded788e3-2bb5-418b-9b5c-3cb76dbb3ecb

In [1]:
import nltk

# Download the necessary datasets and resources
nltk.download('all')  # This downloads all NLTK datasets and resources, which can be large

# Alternatively, download specific datasets
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')


[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     C:\Users\SimWen\AppData\Roaming\nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     C:\Users\SimWen\AppData\Roaming\nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     C:\Users\SimWen\AppData\Roaming\nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_eng to
[nltk_data]    |     C:\Users\SimWen\AppData\Roaming\nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_eng is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     C:\Users\SimWen\AppData\Roaming\nltk_data

True

In [ ]:
# load require package

import pandas as pd
import langchain
from langchain.document_loaders import HuggingFaceDatasetLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
#from transformers import AutoTokenizer, AutoModelForQuestionAnswering
from transformers import AutoTokenizer, pipeline
from langchain import HuggingFacePipeline
from langchain.chains import RetrievalQA

from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from transformers import AutoTokenizer, AutoModelForCausalLM, FalconForQuestionAnswering, FalconConfig
import transformers
import torch

from langchain.vectorstores import VectorStore
import pickle
import pprint

# Set the display options to show all rows and columns
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

## Load FMEA database

In [ ]:
dataset_name = pd.read_csv('output.csv')
dataset_name = dataset_name.astype(object)  # Cast the entire DataFrame to object type
dataset_name.fillna('NA', inplace=True) # fill up na field if any

dataset_name.head(1)

## Preprocess the data
Generate 2 new column : contxt and question

In [ ]:
df_input = pd.DataFrame()

for i in range(len(dataset_name)):

    question_txt = dataset_name.loc[i,'3. Failure cause (FC) of the work element']
    question = "Failure cause of the work element : \n" + question_txt + "\n\n"
    
    failure_mode_txt = dataset_name.loc[i,'2. Failure mode (FM) of the focus element']
    failure_mode = "Failure mode of the focus element : \n" + failure_mode_txt + "\n\n"

    failure_effect_txt = dataset_name.loc[i,'1. Failure effect (FE) to next higher level element and/or end user']
    failure_effect = "Failure effect to next higher level element : \n" + failure_effect_txt + "\n\n"

    current_preventive_control_txt = dataset_name.loc[i,'Current preventive control (PC) for FC']
    current_preventive_control = "Current preventive control : \n" + current_preventive_control_txt + "\n\n"

    current_detection_control_txt = dataset_name.loc[i,'Current detection control (DC) for FC or FM']
    current_detection_control = "Current detection control : \n" + current_detection_control_txt + "\n\n"

    recommended_action_txt = dataset_name.loc[i,'Recommended action']
    recommended_action = "Recommended_action : \n" + recommended_action_txt + "\n\n"

    function_of_process_work_txt = dataset_name.loc[i,'2. Function of the process step and product characteristic']
    function_of_process_work = "Function of the process step : \n" + function_of_process_work_txt + "\n\n"

    function_of_process_step_txt = dataset_name.loc[i,'3. Function of the process work element and process characteristic']
    function_of_process_step = "Function of the process work element : \n" + function_of_process_step_txt + "\n\n"

    location_txt = dataset_name.loc[i,'location']
    sme_remark = "Manufacturing Location : \n" + location_txt + "\n\n"

    sme_remark_txt = dataset_name.loc[i,'SME_Remark']
    sme_remark = "SME Remark : \n" + sme_remark_txt + "\n\n"
    

    # category_txt = dataset_name.loc[i,'5M']
    # category = "5M : \n" + category_txt + "\n\n"
    
    FMEA_txt = dataset_name.loc[i,'fmea_type']
    Name_of_the_unit_process_txt = dataset_name.loc[i,'subject']
    Package_code_txt = dataset_name.loc[i,'package']
    Document_Nr_txt = dataset_name.loc[i,'fmea_id']
    FMEA_idx_id_txt = dataset_name.loc[i,'fmea_idx_id']
    others_info = "Others Info : " + "\n" + "FMEA Type : " +  FMEA_txt +  "\n" + "Name of the unit process : " + Name_of_the_unit_process_txt + "\n" + "Package code : " + Package_code_txt + '\n' + "Manufacturing Location : " + location_txt +"\n" + "FMEA ID : " + Document_Nr_txt + "\n" + "FMEA idx ID : " + FMEA_idx_id_txt                  
    

    
    
    # Create a list of dictionaries with the data
    data = [
        {
            'question': question,
            'context': sme_remark + question +  failure_mode + failure_effect + current_preventive_control + current_detection_control +
                       recommended_action + function_of_process_work + function_of_process_step + others_info,

        }
    ]

    df_data = pd.DataFrame(data)
    df_input = pd.concat([df_input, df_data], axis = 0)


df_input.reset_index(inplace=True)

dataset_name_processed = pd.concat([df_input, dataset_name], axis = 1)
dataset_name_processed.drop('index', axis=1, inplace=True)

dataset_name_processed.head(1)

### Transform dataset_name_processed to jsonl format
so that we can do text spliting embedding and vectorize 

In [ ]:
import pandas as pd
import json
from dataclasses import dataclass

@dataclass
class Document:
    metadata: dict
    page_content: str

In [ ]:
df = dataset_name_processed
documents = []
for index, row in df.iterrows():
    metadata = {
        'question': row['question'],
        '2. Function of the process step and product characteristic' : row['2. Function of the process step and product characteristic'],
        '1. Failure effect (FE) to next higher level element and/or end user': row['1. Failure effect (FE) to next higher level element and/or end user'],
        '2. Failure mode (FM) of the focus element' : row['2. Failure mode (FM) of the focus element'],
        'Current preventive control (PC) for FC' : row['Current preventive control (PC) for FC'],
        'Current detection control (DC) for FC or FM' : row['Current detection control (DC) for FC or FM'],
        'subject' : row['subject'],
        'package': row['package'],
        'location': row['location'],
        'fmeaIid': row['fmea_id'],
        'fmea_idx_id': row['fmea_idx_id']
    }
    page_content = f'"{row["context"]}"'
    document = Document(metadata, page_content)
    documents.append(document)

In [ ]:
# check the transformed document format 
documents[0]

### Text Spliting or chunking

In [ ]:
# Create an instance of the RecursiveCharacterTextSplitter class with specific parameters.
# It splits text into chunks of 1000 characters each with a 150-character overlap.
text_splitter = RecursiveCharacterTextSplitter(chunk_size=3000, chunk_overlap=150)

# 'data' holds the text you want to split, split the text into documents using the text splitter.
docs = text_splitter.split_documents(documents)

In [ ]:
# check the result after text spliting

data = []
for i, chunk in enumerate(docs):
    data.append({'Chunk ID': i, 'Text': chunk})

df = pd.DataFrame(data)
df

### Embeding

In [ ]:

# Define the path to the pre-trained model you want to use
modelPath = "sentence-transformers/all-MiniLM-l6-v2"

# Create a dictionary with model configuration options, specifying to use the CPU for computations
model_kwargs = {'device':'cpu'}

# Create a dictionary with encoding options, specifically setting 'normalize_embeddings' to False
encode_kwargs = {'normalize_embeddings': False}

# Initialize an instance of HuggingFaceEmbeddings with the specified parameters
embeddings = HuggingFaceEmbeddings(
    model_name=modelPath,     # Provide the pre-trained model's path
    model_kwargs=model_kwargs, # Pass the model configuration options
    encode_kwargs=encode_kwargs, # Pass the encoding options
)

In [ ]:
# text = "This is a test document."
# query_result = embeddings.embed_query(text)
# query_result[:3]

### Convert Text To Vector Store

In [ ]:
db = FAISS.from_documents(docs, embeddings)

In [ ]:
# save the db (vector store ) to local
with open("vectorstore_test.pkl", "wb") as f:
    pickle.dump(db, f)

In [ ]:
# load require package

import pandas as pd
import langchain
from langchain.document_loaders import HuggingFaceDatasetLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
#from transformers import AutoTokenizer, AutoModelForQuestionAnswering
from transformers import AutoTokenizer, pipeline
from langchain import HuggingFacePipeline
from langchain.chains import RetrievalQA

from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from transformers import AutoTokenizer, AutoModelForCausalLM, FalconForQuestionAnswering, FalconConfig
import transformers
import torch

from langchain.vectorstores import VectorStore
import pickle
import pprint

# Set the display options to show all rows and columns
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

### load fmea master data

In [ ]:
# selected_row_draft = pd.DataFrame() # initialize selected_row_draft
# dataset_name = pd.read_csv('output.csv')

### Load Vector Store

In [ ]:
# load vectorstore.pkl file from local
vectorstore_file = "vectorstore_test.pkl"

with open(vectorstore_file, "rb") as f:
    global vectorstore
    db: VectorStore = pickle.load(f)

### Configure The LLM
llm part still facing some issue, no solution yet

In [ ]:
# Specify the model name you want to use
#model_name = "Intel/dynamic_tinybert"
model_name = "bert-base-uncased"

# Load the tokenizer associated with the specified model
tokenizer = AutoTokenizer.from_pretrained(model_name, padding=True, truncation=True, max_length=512)

# Define the QA pipeline
qa_pipeline = pipeline('question-answering', 
                       max_length=2000,
                       model=model_name, 
                      tokenizer=tokenizer,
                      return_tensors='pt')


# llm = HuggingFacePipeline(
#     pipeline=question_answerer,
#     model_kwargs={"temperature": 0.7, "max_length": 512},
# )

### Setup The Retrivers

In [ ]:
retriever = db.as_retriever()

In [ ]:
question = "What is the failure effect and failure mode if Unable to transport the leadframe properly resulting to mis index due to worn out guide "
searchDocs = db.similarity_search(question)
for i in range(len(searchDocs)):
    print(searchDocs[i].page_content)
    print("--------------------------------")

In [ ]:
# setup convert text to dictionary function 
def convert_text_to_dict(input_text, question):
    # Remove the leading and trailing delimiters
    clean_text = input_text.strip('------------\n')
    
    # Remove excessive newlines
    clean_text = '\n'.join([line.strip() for line in clean_text.split('\n') if line.strip() != ''])
    
    # Format as multi-line string
    context = f'"""{clean_text}"""'
    
    # Create the dictionary
    result_dict = {
        'context': context,
        'question': question
    }

    return result_dict

In [ ]:
# function to find document
def find_doc(question):
    
    #searchdocs = retriever.invoke(question)
    searchdocs = db.similarity_search(question)
    document_ids = [f"doc_{i}" for i in range(len(searchdocs))]
    
    
    results_list = []
    for i in range(len(document_ids)):
    
        appended_text = searchdocs[i].page_content
    
        result_dict = convert_text_to_dict(appended_text, question)
        
        
        # # Define the QA pipeline
        # qa_pipeline = pipeline('question-answering', 
        #                        max_length=2000,
        #                        model=model_name, 
        #                       tokenizer=tokenizer,
        #                       return_tensors='pt')
        
        # Call the QA model
        result = qa_pipeline(result_dict)
        
    
        # Create a dictionary to store the current result
        result_dict = {
            'Document ID': i,
            'Appended Text': appended_text,
            'Score': result['score'],
            #'Answer': result['answer']
        }
        
        # Append the result dictionary to the results list
        results_list.append(result_dict)

    # Convert the results list to a Pandas DataFrame
    df = pd.DataFrame(results_list)
    df = df.sort_values(by='Score', ascending=False).reset_index(drop=True)

    page_number = df.iloc[0,0]
    fmea_top_score = searchdocs[page_number].page_content
    print(fmea_top_score)

    return df, fmea_top_score






In [ ]:

question = "What is the failure effect and failure mode if Unable to transport the leadframe properly resulting to mis index due to worn out guide "
searchDocs = db.similarity_search(question)
df , fmea_top_score = find_doc(question)
df

In [ ]:
select_this_doc = 'Y'

if select_this_doc == 'Y':
    fmea_idx_id = fmea_top_score.split('FMEA idx ID : ')[1].split('\n')[0].strip('"')
    selected_row = dataset_name[dataset_name['fmea_idx_id'] == fmea_idx_id]
    selected_row_draft = pd.concat([selected_row_draft, selected_row], axis = 0)

else : 
    print('try to change your question')

selected_row_draft

In [ ]:
# export selected doc

selected_row_draft.to_csv('selected_row_draft.csv', index=False, header=True)